**NOTE** This notebook is work under progress

# Interactive exploration of current errors in pandas docstrings

*DISCLAIMER: This notebook is based on the one uploaded by @dujm [here](https://github.com/python-sprints/pandas-mentoring/blob/master/notebooks/docstring_error_interactive.ipynb)*


This notebook will help you detect which errors are still present on some of the docstrings of pandas, so that you can select one of them, fix it, and submit a PR to the [pandas repository](https://github.com/pandas-dev/pandas). 

**IMPORTANT!** Before starting to work on fixing an error, check that nobody is already working on it by searching the issues and PRs in the pandas repository. If you nobody is doing so, open an issue and let others know you will be fixing that docstring.

This script currently supports pandas version >= 0.25.0

Let's start by importing the necessary packages:

In [1]:
import os

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import ipywidgets as widgets
import qgrid

ModuleNotFoundError: No module named 'qgrid'

## *Static exploration*

## 1. Generate a .json containing all current errors

This step was automatically done if you are running this notebook from Binder. Keep in mind that the .json file is updated every 15 minutes, so it might be outdated. When you select an error to work on, double check that nobody has submitted an issue to work on it already.

If you want to generate the .json file locally, simply run the following command from your pandas clone:

`./scripts/validate_docstrings.py --format=json > /path/to/json/pandas_docstring_errors.json`

## 2. Plot a table describing the errors

We will plot a table that indicates which pandas functions still have an error in their docstrings. The specific error code and description will be also described.

In [47]:
# Import JSON as df
file = 'pandas_docstring_errors.json'
df = (pd.read_json(file)
            .transpose()
            .filter(items=['errors', 'file', 'file_line'])
            .explode('errors')
            .dropna()
            .reset_index()
            .rename(columns={"index": "function"})
     )

# Divide errors in their code and description
df[['error_code','error_description']] = pd.DataFrame(df.errors.tolist())
df = df.drop(["errors"], axis=1)

# Print file name relative to pandas repository path
df['file'] = df['file'].str.split('/pandas').str[1]

# Show the ten first examples
df.head(10)

,function,file,file_line,error_code,error_description
0,pandas.BooleanDtype,/core/arrays/boolean.py,39,SA01,See Also section not found
1,pandas.Categorical,/core/arrays/categorical.py,212,PR01,Parameters {'fastpath'} not documented
2,pandas.Categorical,/core/arrays/categorical.py,212,EX02,Examples do not pass tests:\n*****************...
3,pandas.Categorical.__array__,/core/arrays/categorical.py,1271,ES01,No extended summary found
4,pandas.Categorical.__array__,/core/arrays/categorical.py,1271,PR01,Parameters {'dtype'} not documented
5,pandas.Categorical.__array__,/core/arrays/categorical.py,1271,SA01,See Also section not found
6,pandas.Categorical.__array__,/core/arrays/categorical.py,1271,EX01,No examples section found
7,pandas.Categorical.from_codes,/core/arrays/categorical.py,592,SA01,See Also section not found
8,pandas.CategoricalDtype,/core/dtypes/dtypes.py,168,SA04,"Missing description for See Also ""Categorical""..."
9,pandas.CategoricalIndex,/core/indexes/category.py,69,EX02,Examples do not pass tests:\n*****************...


## 3. Count number of functions with errors per error type

In [3]:
df_code = df['error_code'].value_counts().reset_index()
df_code.columns = ['error_code','counts']

df_code

,error_code,counts
0,SA01,376
1,EX01,369
2,ES01,355
3,RT03,296
4,GL08,242
5,EX02,240
6,SA04,239
7,PR01,223
8,PR07,214
9,EX03,144


## 4. Count number of errors per function

In [4]:
df_function = df['function'].value_counts().reset_index()
df_function.columns = ['function','counts']

df_function

,function,counts
0,pandas.PeriodIndex,13
1,pandas.core.groupby.DataFrameGroupBy.boxplot,13
2,pandas.HDFStore.append,13
3,pandas.Series.cat.remove_unused_categories,11
4,pandas.CategoricalIndex.remove_unused_categories,11
...,...,...
1068,pandas.tseries.offsets.CustomBusinessMonthBegi...,1
1069,pandas.tseries.offsets.QuarterOffset.is_anchored,1
1070,pandas.tseries.offsets.MonthEnd.isAnchored,1
1071,pandas.tseries.offsets.Milli.isAnchored,1


## *Interactive exploration* 

Select an error from the following dropdown menu to see a complete description and example of it, and the number of that type of errors in pandas:

In [5]:
# Create dropdown widget
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

dropdown_widget = widgets.Dropdown(options=unique_sorted_values(df_code.error_code),
                                   description='Error:')
dropdown_widget

## TODO: Use observe to print error description

Dropdown(description='Error:', options=('ES01', 'EX01', 'EX02', 'EX03', 'GL01', 'GL08', 'PR01', 'PR02', 'PR06'…

You can filter the following table by the error code you want to work on, or its function.

In [6]:
# Create qgrid widget
qgrid_widget = qgrid.show_grid(df, grid_options={'forceFitColumns': True})
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…